In [2]:
from transformers import (
    GPT2PreTrainedModel, 
    GPT2Config, 
    GPT2Model, 
    GPT2TokenizerFast, 
    DataCollatorForLanguageModeling
)
from transformers import Trainer, TrainingArguments
from transformers.models.gpt2.modeling_gpt2 import GPT2Block
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions
)
from tokenizers import Tokenizer
from torch import nn
from torch.utils.data import Dataset
from pathlib import Path
import torch
from packaging import version
from tqdm import tqdm
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
import wandb

In [ ]:
wandb.login()

In [16]:
TOKENIZER_SAVEDIR = Path('/home/macosta/ttmp/primus-data/primus-semantic/semantic-tokenizer-v3/')
LM_MODEL_SAVEDIR = Path('/home/macosta/ttmp/primus-models/gpt2-lm-semantic-rhythm-more/')
Path(LM_MODEL_SAVEDIR).mkdir(exist_ok=True)
TXT_FILES = Path('/home/macosta/ttmp/primus-data/primus-semantic/semantic-cleaned-v3')

In [5]:
class CustomGPT2Embeddings(nn.Module):
    """Construct the embeddings from token, rhythmic position, and position."""

    def __init__(self, config):
        super().__init__()
        self.sixteenths_per_bar = 0
        self.sixteenths_left = 0
        self.idx_to_action = config.idx_to_action
        self.last_duration = 0
        self.n_rhythmic_positions = config.n_rhythmic_positions
        
        #Declare embedding layers
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.rhythm_position_embeddings = nn.Embedding(self.n_rhythmic_positions, config.hidden_size)
        print(f"Rhythmic position embedding shape: {self.rhythm_position_embeddings.weight.shape}")
        self.weight = self.token_embeddings.weight

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
#         self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))
        if version.parse(torch.__version__) > version.parse("1.6.0"):
            self.register_buffer(
                "token_type_ids",
                torch.zeros(self.position_ids.size(), dtype=torch.long),
                persistent=False,
            )
            
    def update_state(self, token_idx):
        action, data = self.idx_to_action[token_idx.item()]
        if action == "RESET":
            self.sixteenths_left = self.sixteenths_per_bar 
            self.last_duration = 0
        elif action == "USE_LAST_DURATION":
            self.sixteenths_left -= int(data * self.last_duration)
        elif action == "SET_TIMESIG":
            self.sixteenths_per_bar = data
            self.sixteenths_left = data
        else:
            self.sixteenths_left -= data
            self.last_duration = data
        self.sixteenths_left = max(self.sixteenths_left, 0)
            
    def forward(
        self, input_ids=None, position_ids=None
    ):
        #Custom code to use 2 embedding layers
        batch_tokens = []
        batch_rhythms = []
        #Iterate through the batch
        for sequence in input_ids:
            #For each sequence, make a list to store the octave, pitch, and handConf ids
            tokens = []
            rhythms = []
            self.sixteenths_left = self.sixteenths_per_bar
            #Iterate through the sequence
            for token_idx in sequence:
                self.update_state(token_idx)
                #Add each element to the correct list
                tokens.append(token_idx)
                rhythms.append(min(int(self.sixteenths_left), self.n_rhythmic_positions - 1))
            #Aggregate the samples in the batch
            batch_tokens.append(tokens)
            batch_rhythms.append(rhythms)
            
        device = input_ids.device if input_ids is not None else inputs_embeds.device
        
        #Convert the lists to tensors and put them on the gpu
        tokenTensor = torch.LongTensor(batch_tokens).to(device)
        rhythmTensor = torch.LongTensor(batch_rhythms).to(device)
        
        #Sum the two embeddings
        input_embeds = self.token_embeddings(tokenTensor)\
                       +self.rhythm_position_embeddings(rhythmTensor)
        embeddings = input_embeds

        #Standard BertEmbeddings code
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]
            raise Exception('Input ids should not be None')

        seq_length = input_shape[1]
        
        if position_ids is None:
            position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).expand(input_shape)
            raise Exception('Input ids should not be None')
        position_embeddings = self.position_embeddings(position_ids)
        embeddings += position_embeddings
#         embeddings = self.LayerNorm(embeddings)
#         embeddings = self.dropout(embeddings)
        return embeddings

In [6]:
class CustomGPT2Model(GPT2PreTrainedModel):
    _keys_to_ignore_on_load_missing = ["attn.masked_bias"]

    def __init__(self, config):
        super().__init__(config)

        self.embed_dim = config.hidden_size

        ''' OLD '''
#         self.wte = nn.Embedding(config.vocab_size, self.embed_dim)
#         self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)
        ''' NEW '''
        self.embeddings = CustomGPT2Embeddings(config)

        self.drop = nn.Dropout(config.embd_pdrop)
        self.h = nn.ModuleList([GPT2Block(config, layer_idx=i) for i in range(config.num_hidden_layers)])
        self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)

        # Model parallel
        self.model_parallel = False
        self.device_map = None
        self.gradient_checkpointing = False

        # Initialize weights and apply final processing
        self.post_init()

    def parallelize(self, device_map=None):
        # Check validity of device_map
        self.device_map = (
            get_device_map(len(self.h), range(torch.cuda.device_count())) if device_map is None else device_map
        )
        assert_device_map(self.device_map, len(self.h))
        self.model_parallel = True
        self.first_device = "cpu" if "cpu" in self.device_map.keys() else "cuda:" + str(min(self.device_map.keys()))
        self.last_device = "cuda:" + str(max(self.device_map.keys()))
        ''' OLD '''
#         self.wte = self.wte.to(self.first_device)
#         self.wpe = self.wpe.to(self.first_device)
        ''' NEW '''
        self.embeddings = self.embeddings.to(self.first_device)
    
        # Load onto devices
        for k, v in self.device_map.items():
            for block in v:
                cuda_device = "cuda:" + str(k)
                self.h[block] = self.h[block].to(cuda_device)
        # ln_f to last
        self.ln_f = self.ln_f.to(self.last_device)

    def deparallelize(self):
        self.model_parallel = False
        self.device_map = None
        self.first_device = "cpu"
        self.last_device = "cpu"
        ''' OLD '''
#         self.wte = self.wte.to("cpu")
#         self.wpe = self.wpe.to("cpu")
        ''' NEW '''
        self.embeddings = self.embedding.to("cpu")
        
        for index in range(len(self.h)):
            self.h[index] = self.h[index].to("cpu")
        self.ln_f = self.ln_f.to("cpu")
        torch.cuda.empty_cache()

    def get_input_embeddings(self):
        ''' OLD '''
#         return self.wte
        ''' NEW '''
        return self.embeddings

    def set_input_embeddings(self, new_embeddings):
        ''' OLD '''
#         self.wte = new_embeddings
        ''' NEW '''
        self.embeddings = new_embeddings

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer}
        """
        for layer, heads in heads_to_prune.items():
            self.h[layer].attn.prune_heads(heads)

    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
            input_ids = input_ids.view(-1, input_shape[-1])
            batch_size = input_ids.shape[0]
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
            batch_size = inputs_embeds.shape[0]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        if token_type_ids is not None:
            token_type_ids = token_type_ids.view(-1, input_shape[-1])
        if position_ids is not None:
            position_ids = position_ids.view(-1, input_shape[-1])

        if past_key_values is None:
            past_length = 0
            past_key_values = tuple([None] * len(self.h))
        else:
            past_length = past_key_values[0][0].size(-2)
        if position_ids is None:
            position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])

        # GPT2Attention mask.
        if attention_mask is not None:
            if batch_size <= 0:
                raise ValueError("batch_size has to be defined and > 0")
            attention_mask = attention_mask.view(batch_size, -1)
            # We create a 3D attention mask from a 2D tensor mask.
            # Sizes are [batch_size, 1, 1, to_seq_length]
            # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
            # this attention mask is more simple than the triangular masking of causal attention
            # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
            attention_mask = attention_mask[:, None, None, :]

            # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
            # masked positions, this operation will create a tensor which is 0.0 for
            # positions we want to attend and -10000.0 for masked positions.
            # Since we are adding it to the raw scores before the softmax, this is
            # effectively the same as removing these entirely.
            attention_mask = attention_mask.to(dtype=self.dtype)  # fp16 compatibility
            attention_mask = (1.0 - attention_mask) * -10000.0

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.add_cross_attention and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # head_mask has shape n_layer x batch x n_heads x N x N
        head_mask = self.get_head_mask(head_mask, self.config.n_layer)

        ''' OLD '''
#         if inputs_embeds is None:
#             inputs_embeds = self.wte(input_ids)
#         position_embeds = self.wpe(position_ids)
#         hidden_states = inputs_embeds + position_embeds
#         if token_type_ids is not None:
#             token_type_embeds = self.wte(token_type_ids)
#             hidden_states = hidden_states + token_type_embeds
        ''' NEW '''
        if input_ids is None:
            raise Exception("Need input ids")
        hidden_states = self.embeddings(input_ids, position_ids)

        hidden_states = self.drop(hidden_states)

        output_shape = input_shape + (hidden_states.size(-1),)

        presents = () if use_cache else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None
        all_hidden_states = () if output_hidden_states else None
        for i, (block, layer_past) in enumerate(zip(self.h, past_key_values)):

            # Model parallel
            if self.model_parallel:
                torch.cuda.set_device(hidden_states.device)
                # Ensure layer_past is on same device as hidden_states (might not be correct)
                if layer_past is not None:
                    layer_past = tuple(past_state.to(hidden_states.device) for past_state in layer_past)
                # Ensure that attention_mask is always on the same device as hidden_states
                if attention_mask is not None:
                    attention_mask = attention_mask.to(hidden_states.device)
                if isinstance(head_mask, torch.Tensor):
                    head_mask = head_mask.to(hidden_states.device)
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        # None for past_key_value
                        return module(*inputs, use_cache, output_attentions)

                    return custom_forward

                outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(block),
                    hidden_states,
                    None,
                    attention_mask,
                    head_mask[i],
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                outputs = block(
                    hidden_states,
                    layer_past=layer_past,
                    attention_mask=attention_mask,
                    head_mask=head_mask[i],
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    use_cache=use_cache,
                    output_attentions=output_attentions,
                )

            hidden_states = outputs[0]
            if use_cache is True:
                presents = presents + (outputs[1],)

            if output_attentions:
                all_self_attentions = all_self_attentions + (outputs[2 if use_cache else 1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (outputs[3 if use_cache else 2],)

            # Model Parallel: If it's the last layer for that device, put things on the next device
            if self.model_parallel:
                for k, v in self.device_map.items():
                    if i == v[-1] and "cuda:" + str(k) != self.last_device:
                        hidden_states = hidden_states.to("cuda:" + str(k + 1))

        hidden_states = self.ln_f(hidden_states)

        hidden_states = hidden_states.view(*output_shape)
        # Add last hidden state
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [hidden_states, presents, all_hidden_states, all_self_attentions, all_cross_attentions]
                if v is not None
            )

        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )

In [7]:
class CustomGPT2LMHeadModel(GPT2PreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"attn.masked_bias", r"attn.bias", r"lm_head.weight"]

    def __init__(self, config):
        super().__init__(config)
        self.transformer = CustomGPT2Model(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # Model parallel
        self.model_parallel = False
        self.device_map = None

        # Initialize weights and apply final processing
        self.post_init()

    def parallelize(self, device_map=None):
        self.device_map = (
            get_device_map(len(self.transformer.h), range(torch.cuda.device_count()))
            if device_map is None
            else device_map
        )
        assert_device_map(self.device_map, len(self.transformer.h))
        self.transformer.parallelize(self.device_map)
        self.lm_head = self.lm_head.to(self.transformer.first_device)
        self.model_parallel = True

    def deparallelize(self):
        self.transformer.deparallelize()
        self.transformer = self.transformer.to("cpu")
        self.lm_head = self.lm_head.to("cpu")
        self.model_parallel = False
        torch.cuda.empty_cache()

    def get_output_embeddings(self):
        return self.lm_head

    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings

    def prepare_inputs_for_generation(self, input_ids, past=None, **kwargs):
        token_type_ids = kwargs.get("token_type_ids", None)
        # only last token for inputs_ids if past is defined in kwargs
        if past:
            input_ids = input_ids[:, -1].unsqueeze(-1)
            if token_type_ids is not None:
                token_type_ids = token_type_ids[:, -1].unsqueeze(-1)

        attention_mask = kwargs.get("attention_mask", None)
        position_ids = kwargs.get("position_ids", None)

        if attention_mask is not None and position_ids is None:
            # create position_ids on the fly for batch generation
            position_ids = attention_mask.long().cumsum(-1) - 1
            position_ids.masked_fill_(attention_mask == 0, 1)
            if past:
                position_ids = position_ids[:, -1].unsqueeze(-1)
        else:
            position_ids = None
        return {
            "input_ids": input_ids,
            "past_key_values": past,
            "use_cache": kwargs.get("use_cache"),
            "position_ids": position_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }
    
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for language modeling. Note that the labels **are shifted** inside the model, i.e. you can set
            `labels = input_ids` Indices are selected in `[-100, 0, ..., config.vocab_size]` All labels set to `-100`
            are ignored (masked), the loss is only computed for labels in `[0, ..., config.vocab_size]`
        """
#         print(labels)
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = transformer_outputs[0]

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.transformer.first_device)
            hidden_states = hidden_states.to(self.lm_head.weight.device)

        ''' OLD '''
        lm_logits = self.lm_head(hidden_states)
        ''' NEW '''
#         rhythm_p, token_p = self.lm_head(hidden_states)

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            ''' OLD '''
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            ''' NEW '''
#             shift_rhythm_logits = rhythm_p[..., :-1, :].contiguous()
#             shift_token_logits = token_p[..., :-1, :].contiguous()
#             shift_labels = labels[..., 1:].contiguous()
            
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return CausalLMOutputWithCrossAttentions(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
            cross_attentions=transformer_outputs.cross_attentions,
        )

In [8]:
class CustomGPT2Config(GPT2Config):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.idx_to_action = kwargs.get('idx_to_action')
        self.n_rhythmic_positions = kwargs.get('n_rhythmic_positions')

In [9]:
def get_sixteenths(token):
    if token == 'quadruple_whole':
        return 64
    elif token == 'double_whole':
        return 32
    elif token == 'whole':
        return 16
    elif token == 'half':
        return 8
    elif token == 'quarter':
        return 4
    elif token == 'eighth':
        return 2
    elif token == 'sixteenth':
        return 1
    elif token == 'thirty_second':
        return 0.5
    elif token == 'sixty_fourth':
        return 0.25
    return 0

In [10]:
def parse_time_sig(time_sig):
    time_sig = time_sig.split('-')[1]
    if time_sig == 'C':
        return 4, 4
    elif time_sig == 'C/':
        return 2, 2
    elif '/' in time_sig:
        top, bottom = time_sig.split('/')
        return int(top), int(bottom)
    else:
        return 4, 4

In [11]:
def time_sig_to_sixteenths(time_sig):
    top, bottom = parse_time_sig(time_sig)
    return (16 // bottom) * top

In [13]:
def token_to_action(token):
    if token in ['barline', '</s>', '<s>']:
        return ("RESET", 0)
    elif token == 'dot':
        return ("USE_LAST_DURATION", 0.5)
    elif token == 'dotdot':
        return ("USE_LAST_DURATION", 0.25)
    elif len(token) > 14 and token[:14] == 'timeSignature-':
        return ("SET_TIMESIG", time_sig_to_sixteenths(token))
    else:
        return ("DECREMENT", get_sixteenths(token))

In [17]:
class CustomDataset(Dataset):
    def __init__(self, src_files, tokenizer, max_length):
        self.examples = []
        for src_file in tqdm(src_files):
            words = src_file.read_text(encoding="utf-8")
            words = words.split()
            if 'thirty_second' in words or 'sixty_fourth' in words:
                continue
            words = ['<s>'] + words + ['</s>']
            for i in range(0, len(words), max_length):
                word_string = ' '.join(words[i:i+max_length])
                tokenized = tokenizer.encode(word_string, max_length=max_length, padding='max_length')
                assert(len(tokenized) == max_length)
                self.examples.append(tokenized)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i])

In [15]:
def create_train_test_datasets(tokenizer, max_length, fraction=1.0, test_size=0.1):
    src_files = list(Path(TXT_FILES).glob("**/*.semantic"))
    src_files = src_files[:int(len(src_files) * fraction)]
    split_index = int(len(src_files) * (1 - test_size))
    train_files = src_files[:split_index]
    test_files = src_files[split_index:]
    train_dataset = CustomDataset(train_files, tokenizer, max_length=max_length)
    test_dataset = CustomDataset(test_files, tokenizer, max_length=max_length)
    return train_dataset, test_dataset

In [14]:
temp_tokenizer = Tokenizer.from_file(str(TOKENIZER_SAVEDIR / 'tokenizer.json'))
tokenizer = GPT2TokenizerFast(tokenizer_object=temp_tokenizer, 
                                         unk_token='<unk>',
                                         pad_token='<pad>',
                                         bos_token='<s>',
                                         eos_token='</s>')

In [15]:
ACTUAL_VOCAB_SIZE = len(tokenizer.vocab)
MAX_LEN=128

In [16]:
train_dataset, test_dataset = create_train_test_datasets(tokenizer, MAX_LEN, fraction=1, test_size=0.05)

100%|█████████████████████████████████████| 4384/4384 [00:00<00:00, 4543.37it/s]


In [18]:
tokenizer.decode(train_dataset.__getitem__(10))

'<s> clef-G2 keySignature-GM timeSignature-3/4 note D5 eighth dot note E5 sixteenth barline note E5 quarter dot note D5 eighth note D5 eighth dot note E5 sixteenth barline note E5 quarter note D5 quarter note D5 eighth note D5 eighth barline note G5 quarter dot note B4 eighth note F#5 eighth dot note E5 sixteenth barline note D5 quarter note C#5 quarter </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [20]:
index_to_action = [token_to_action(tokenizer.decode(i)) for i in range(len(tokenizer.vocab))]

In [21]:
for i in range(10, 30):
    print(tokenizer.decode(i), index_to_action[i])

rest ('DECREMENT', 0)
half ('DECREMENT', 8)
C5 ('DECREMENT', 0)
A4 ('DECREMENT', 0)
dot ('USE_LAST_DURATION', 0.5)
G4 ('DECREMENT', 0)
E5 ('DECREMENT', 0)
B4 ('DECREMENT', 0)
Bb4 ('DECREMENT', 0)
G5 ('DECREMENT', 0)
F5 ('DECREMENT', 0)
clef-G2 ('DECREMENT', 0)
D4 ('DECREMENT', 0)
F4 ('DECREMENT', 0)
E4 ('DECREMENT', 0)
thirty_second ('DECREMENT', 0.5)
C#5 ('DECREMENT', 0)
timeSignature-C ('SET_TIMESIG', 16)
F#5 ('DECREMENT', 0)
A5 ('DECREMENT', 0)


In [22]:
config = CustomGPT2Config(
    vocab_size=ACTUAL_VOCAB_SIZE,
    bos_token_id=0,
    eos_token_id=2,
    n_rhythmic_positions=32 + 1,
    idx_to_action=index_to_action,
    n_positions=MAX_LEN
)

In [27]:
model = CustomGPT2LMHeadModel(config=config)

In [24]:
print(model.transformer.embeddings.rhythm_position_embeddings.weight[6,:100])

tensor([-0.0090, -0.0363,  0.0116, -0.0006,  0.0072, -0.0320,  0.0118, -0.0363,
        -0.0138, -0.0087, -0.0335, -0.0174,  0.0283,  0.0006, -0.0002,  0.0205,
        -0.0043,  0.0182,  0.0010,  0.0018, -0.0297,  0.0058,  0.0179,  0.0017,
        -0.0144,  0.0024, -0.0050,  0.0150, -0.0027, -0.0013, -0.0011,  0.0029,
         0.0046, -0.0237, -0.0218,  0.0102,  0.0118, -0.0105,  0.0237,  0.0153,
        -0.0091,  0.0114,  0.0035,  0.0283,  0.0185,  0.0017, -0.0080,  0.0213,
         0.0206,  0.0247, -0.0226,  0.0058, -0.0266, -0.0212, -0.0263, -0.0127,
         0.0044,  0.0098, -0.0314, -0.0119, -0.0110, -0.0212,  0.0197, -0.0156,
        -0.0338,  0.0027,  0.0271,  0.0066, -0.0111,  0.0067,  0.0007, -0.0011,
         0.0116,  0.0116,  0.0144, -0.0185, -0.0012, -0.0030,  0.0220, -0.0039,
        -0.0052, -0.0217,  0.0197, -0.0061, -0.0309,  0.0311, -0.0147,  0.0174,
         0.0113, -0.0130,  0.0282, -0.0113, -0.0055, -0.0038,  0.0014, -0.0093,
        -0.0044,  0.0339,  0.0158, -0.02

In [28]:
training_args = TrainingArguments(
    output_dir=LM_MODEL_SAVEDIR,
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=64,
    save_steps=10000,
    logging_steps=3000,
    evaluation_strategy="steps",
    eval_steps=3000,
    save_total_limit=1,
    prediction_loss_only=False,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

PyTorch: setting up devices


In [29]:
ret = trainer.train()

/home/macosta/anaconda3/envs/mir2/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 83399
  Num Epochs = 15
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 19560
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: macosta (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/macosta/anaconda3/envs/mir2/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
print(model.transformer.embeddings.rhythm_position_embeddings.weight[6,:100])

In [ ]:
trainer.save_model(LM_MODEL_SAVEDIR)